In [17]:
import pandas as pd
import tqdm
import requests
from datasets import Dataset
import os
import getpass
from clearml import Task

from ragas.metrics import (
    answer_relevancy,
    faithfulness,
    context_recall,
    context_precision,
)

from ragas import evaluate

In [68]:
os.environ["OPENAI_API_KEY"] = getpass.getpass('Enter your key: ')

In [19]:
# clear ml credentials

In [20]:
test_data_path = "/Users/ekaterina.baru/projects/SpicyRAG/data/eval_dataset.xlsx"

In [21]:
# project:
#   name: "SpicyRag"
# 
# database:
#   user: "user"
#   password: "password"
#   name: "spicyragdb"
#   host: "localhost"
#   port: 5432
# 
# logging:
#   level: "INFO"
# 
# server:
#   host: "0.0.0.0"
#   port: 8000
# 
# llm:
#   model: "google/gemma-2-27b-it"
#   role: "user"
#   system_prompt: |
#     Вы являетесь виртуальным юридическим консультантом, специализирующимся на анализе и интерпретации юридических документов и правовых норм.
#     Ваша задача — помочь пользователю, используя предоставленный контекст, чтобы дать точный и обоснованный ответ на его юридический вопрос.
#     Контекст может содержать выдержки из законов, нормативных актов, судебных решений, договоров или других юридических текстов.
# 
#     Инструкции:
#       1. Внимательно проанализируй контекст и определите, какие части контекста наиболее релевантны, приводи всю полезную информацию из контекста
#       2. Используй релевантные фрагменты контекста (не пиши к чему обращаешься, не упоминай пунктов) для создания полного и юридически обоснованного ответа. Приводи цитаты из контекста в соответствии
#       3. После ответа напиши вывод о написанной тобой информации, начиная со слова Таким образом
# 
#     ВАЖНО: не упоминай, что ты работаешь с контекстом! Отвечай структурно и по пунктам.
#   temperature: 0.15
#   top_p: 0.7
#   max_tokens: 4096
# 
# llm_rewriter:
#   model: "meta/llama-3.1-405b-instruct"
#   role: "user"
#   system_prompt: |
#     Вы - полезный помощник, который генерирует несколько поисковых запросов на основе одного входного запроса.
#     Выполните расширение запроса. Если существует несколько распространенных способов формулировки пользовательского вопроса или общих синонимов для ключевых слов в вопросе, обязательно верните несколько версий запроса с разными формулировками.
#     Если в тексте есть сокращения или слова, которые вам незнакомы, не пытайтесь их перефразировать.
#     Верните только 3 разных варианта вопроса.
#   temperature: 0.2
#   top_p: 0.7
#   max_tokens: 2048
# 
# embedding_model:
#   name: "paraphrase-MiniLM-L6-v2"
#   dimension: 384
# 
# data_sources:
#   excel_file: "./data/v2_ragas_npa_dataset_firstPart.xlsx"
#   text_file: "./data/hmao_npa.txt"
#   text_separator: "\n\n"
# 
# data_processing:
#   chunker:
#     py_class: interface.chunker.RecursiveCharacterTextSplitterChunker
#     kwargs:
#       chunk_size: 2048
#       chunk_overlap: 128
#       separators:
#         - '\n#{1,6} '
#         - '```\n'
#         - '\n\\*\\*\\*+\n'
#         - '\n---+\n'
#         - '\n___+\n'
#         - '\n\n'
#         - '\n'
#         - ' '
#         - ''
#         - ','
#         - '|'
#   top_k: 3
#   similarity_threshold: 0.7

In [62]:
def get_system_responses(questions):
    """
    Send requests to the /ask endpoint and collect responses.
    """
    responses = []
    for question in tqdm.tqdm(questions.itertuples()):
        try:
            response = requests.post("http://localhost:8000/ask/", json={"question": question[2]})
            response.raise_for_status()
            response_data = response.json()
            responses.append(
                {
                    "question": question[2],
                    "response": response_data["response"],
                    "ground_truth": question[4],
                    "context": response_data["context"], 
                }
            )
        except Exception as e:
            print(f"Error while sending query: {question[1]}, error message: {e}")
            continue
    return responses

In [63]:
test_data = pd.read_excel(test_data_path)

In [64]:
system_responses = get_system_responses(test_data)

120it [08:43,  4.36s/it]


In [66]:
data_samples = {
    "question": [response["question"] for response in system_responses],
    "answer": [response["response"] for response in system_responses],
    "contexts": [response["context"] for response in system_responses],
    "ground_truth": [response["ground_truth"] for response in system_responses],
}
dataset = Dataset.from_dict(data_samples)

In [69]:
result = evaluate(
    dataset,
    metrics=[
        context_precision,
        faithfulness,
        answer_relevancy,
        context_recall,
    ],
)

result

Evaluating:   0%|          | 0/480 [00:00<?, ?it/s]

Exception raised in Job[275]: RateLimitError(Error code: 429 - {'error': {'message': 'Rate limit reached for gpt-4o-mini in organization org-UZZpjHJPzrDNOTXMz3GCGmjK on tokens per min (TPM): Limit 200000, Used 197950, Requested 5343. Please try again in 987ms. Visit https://platform.openai.com/account/rate-limits to learn more.', 'type': 'tokens', 'param': None, 'code': 'rate_limit_exceeded'}})
Exception raised in Job[279]: RateLimitError(Error code: 429 - {'error': {'message': 'Rate limit reached for gpt-4o-mini in organization org-UZZpjHJPzrDNOTXMz3GCGmjK on tokens per min (TPM): Limit 200000, Used 196467, Requested 5022. Please try again in 446ms. Visit https://platform.openai.com/account/rate-limits to learn more.', 'type': 'tokens', 'param': None, 'code': 'rate_limit_exceeded'}})
Exception raised in Job[267]: RateLimitError(Error code: 429 - {'error': {'message': 'Rate limit reached for gpt-4o-mini in organization org-UZZpjHJPzrDNOTXMz3GCGmjK on tokens per min (TPM): Limit 200000,

{'context_precision': 0.8018, 'faithfulness': 0.4913, 'answer_relevancy': 0.0303, 'context_recall': 0.4853}

In [31]:
# {'context_precision': 0.8018, 'faithfulness': 0.4913, 'answer_relevancy': 0.0303, 'context_recall': 0.4853}

In [71]:
# Initialize the task
task = Task.init(
    project_name="SpicyRAG_experiments_tracker", 
    task_name="Exp_2_Gemma",  
    task_type=Task.TaskTypes.application  # Specify the task type
)

# Log each metric
for metric_name, metric_value in result.items():
    task.get_logger().report_scalar(
        title=metric_name,   # Metric name
        series="evaluation", # Group or series (optional but useful for grouping related metrics)
        value=metric_value,  # The value of the metric
        iteration=1          # The iteration number or epoch (set accordingly)
    )

# Close the task when done
task.close()

ClearML Task: created new task id=fd57f536db844e0d904ad189b244104a
2024-09-07 09:56:28,266 - clearml.Task - INFO - Storing jupyter notebook directly as code
ClearML results page: https://app.clear.ml/projects/29330718a2a04a3582ac1092ef326bea/experiments/fd57f536db844e0d904ad189b244104a/output/log
2024-09-07 09:56:30,783 - clearml.Task - INFO - Waiting for repository detection and full package requirement analysis
2024-09-07 09:56:32,632 - clearml.Task - INFO - Finished repository detection and package analysis


In [75]:
import json

with open("exp_2_gemma.json", 'w') as f:
    json.dump(system_responses, f, indent=4) 